<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Capstone: Shopee Sales Analysis

## Executive Summary
[*Jump to table of contents*](#Table-of-Contents)

In this analysis, we will be using the pycaret library to get the best models to predict sales performance in terms of quantity on Shopee. This project be focused on the Beauty and Personal Care category. This will not only allow us to forecast future sales performances in terms of categories, and will help sellers remain competitive on the platform. This in turn, increases sellers' efficiency on the platform.

We will be scraping data using Selenium from shops mainly on Shopee Mall, as it is an area on Shopee that lists verified sellers - mainly authorised distributors, retailers and flagship stores. These sellers are all regulated and are required to sell authentic products, following the guidelines of the brands they are selling. We will focus mainly on Watsons Official Store as it has a huge variety of 3000 product listings from different brands and categories within the health and beauty category, which provides us with a substantial sample size. 

The provided data dictionary is reflected below, along with an additional column describing the corresponding data type.

**The main metric we will be using to assess the models is RMSE (root mean squared error) as it allows us to accurately assess how a model performs both on train and test data. Although we are optimizing for RMSE, we will also be looking at R2 to compare between models. 

This analysis will benefit retailers in the Health and Beauty industry to make better business decisions and pick more popular products to increase sales. On Shopee, we can see that there is a limit of 3000 product listings, and larger distributors and retailers will have more than 3000 products. Listing products online, apart from their brick and mortar stores requires additional costs (creation of listings, allocating stocks to e-commerce, online stock inventory system etc.). Hence this will help Health and Beauty retailers have a better selection of products from popular categories, meeting the needs of shoppers.

In this analysis, we will be looking to answer the following questions:
* What features are most relevant for sellers to increase sale quantity?
* In what ways do categories or sub categories impact sales quantity?
* What mechanics are ideal for their range of products? 
* What other insights can we get from the analysis? 


## Problem Statement
This project aims to help retailers in the Health and Beauty industry by finding out key features or criterias that affect sale quantity on e-commerce platform. This allows sellers to have better selections of products that better suit the needs of shoppers on Shopee. 

The model can also be used to forecast future sales, and hence allow brands to restock sufficient quantities to meet the volume on the platform. This prevents sellers and brands from missing out on sales.

The model will be trained using products from Watsons' official store on Shopee Mall, using the PyCaret which will determine the suitable models. 

## Table of Contents

* [i. Data Dictionary](#i.-Data-Dictionary)
    * [1. Data Cleaning](#1.-Data-Cleaning)
        * [1.1 Installing PyCaret](#1.1-Installing-PyCaret)
        * [1.2 Import Libraries](#1.2-Import-Libraries)
        * [1.3 Preparing Dataset for Modelling](#1.3-Preparing-Dataset-for-Modelling)
    * [2. Modelling](#2.-Modelling)
        * [2.1 PyCaret for Baseline Model](#2.1-PyCaret-for-Baseline-Model)
            * [1.1.5 Scraping earliest review dates from each product link](#1.1.5-Scraping-earliest-review-dates-from-each-product-link)

## 1. Data Cleaning
[*Jump to table of contents*](#Table-of-Contents)
### 1.1 Installing PyCaret
[*Jump to table of contents*](#Table-of-Contents)

In [1]:
# !pip install pycaret shap # Only run once

In [2]:
# !pip uninstall -y scikit-learn 
# !pip install scikit-learn==0.23.2 # Only run once

In [3]:
# check version
from pycaret.utils import version
version()

'2.3.5'

### 1.2 Import Libraries
[*Jump to table of contents*](#Table-of-Contents)

In [4]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.regression import *

plt.rcParams['font.size'] = 14

sns.set_style(style='white')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
# load CSV file as df
df = pd.read_csv('watsons_dclean.csv')

### 1.3 Preparing Dataset for Modelling
[*Jump to table of contents*](#Table-of-Contents)

In [7]:
df

,Unnamed: 0,ProductName,OriginalPrice,SalePrice,Discount,Vouchers,Ratings,NoOfRatings,Comments,Media,Brand,QtySold,Favourites,ProductDescription,link,SubCat1,SubCat2,SubCat3,Content
0,0,SD BIOSENSOR Standard Q Covid-19 AG Home Test ...,13.00,7.30,44,None,4.9,830,424,312,SD BIOSENSOR,16500,1700,SD BIOSENSOR Standard Q Covid-19 AG Home Test ...,https://shopee.sg/SD-BIOSENSOR-Standard-Q-Covi...,Health & Wellness,Medical Supplies,Health Monitors,276.0
1,1,Kodomo Baby Wipes 70'S,2.00,2.00,0,None,5.0,251,96,61,KODOMO,4200,222,Watsons is Singapore's leading beauty and heal...,https://shopee.sg/Kodomo-Baby-Wipes-70'S-i.195...,"Toys, Kids & Babies",Bath & Baby Care,Baby Wipes,296.0
2,2,Dr Tung's Smart Dental Floss 27M,5.10,4.80,6,"$6 OFF, $12 OFF, $25 OFF",5.0,70,32,19,Dr.Tung's,648,68,Watsons is Singapore's leading beauty and heal...,https://shopee.sg/Dr-Tung's-Smart-Dental-Floss...,Beauty & Personal Care,Oral Care,Oral Accessories,296.0
3,3,Watsons Goat Milk Cream Bath 1L,6.90,2.45,64,"$6 OFF, $12 OFF, $25 OFF",5.0,154,59,53,Watsons,606,76,Watsons is Singapore's leading beauty and heal...,https://shopee.sg/Watsons-Goat-Milk-Cream-Bath...,Beauty & Personal Care,Skincare,Moisturizer,296.0
4,4,WATSONS Square Puffs Facial Cotton - 3x160s (E...,4.90,4.90,0,"$6 OFF, $12 OFF, $25 OFF",5.0,143,51,44,Watsons,439,96,"Watsons Square Puffs are made from lint free, ...",https://shopee.sg/WATSONS-Square-Puffs-Facial-...,Beauty & Personal Care,Beauty Devices & Tools,Makeup Accessories,519.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,2921,Durex Extra Safe 3s,6.25,6.25,0,"$6 OFF, $12 OFF, $25 OFF",5.0,1,0,0,durex,3,0,Slightly thicker than our standard condoms for...,https://shopee.sg/Durex-Extra-Safe-3s-i.195238...,Health & Wellness,Sexual Wellness,Condoms,1098.0
2922,2922,NATURE'S ESSENCE Brite Eyes Vegetarian Caplets...,36.45,36.45,0,"$6 OFF, $12 OFF, $25 OFF",5.0,1,1,1,NATURE'S ESSENCE,1,0,Brite Eyes contains a comphrensive formula tha...,https://shopee.sg/NATURE'S-ESSENCE-Brite-Eyes-...,Health & Wellness,Food Nutrition,Others,992.0
2923,2923,Kinohimitsu D'Tox Plum Juice (Flush Out Toxins...,39.90,33.85,15,"$6 OFF, $12 OFF, $25 OFF",5.0,1,1,1,KINOHIMITSU,1,0,"""KINOHIMITSU D’tox Juice 6-Day Program is a sm...",https://shopee.sg/Kinohimitsu-D'Tox-Plum-Juice...,Health & Wellness,Weight Management,Others,793.0
2924,2924,Leaders Insolution Mediu Amino Lifting Mask 5S,12.00,10.20,15,"$6 OFF, $12 OFF, $25 OFF",5.0,1,0,0,LEADERS,1,1,Mediu Amino Lifting Mask firms up your skin by...,https://shopee.sg/Leaders-Insolution-Mediu-Ami...,Beauty & Personal Care,Skincare,Mask,1687.0


In [8]:
#Remove Unnamed: 0 - it is a index column
df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
df.shape # Check if the column is dropped

(2926, 18)

In [9]:
#Remove ProductDescription as we will not be analysing long texts
df.drop(columns=['ProductDescription'], axis=1, inplace=True)
df.shape # Check if the column is dropped

(2926, 17)

In [10]:
#Remove ProductName and link as we will not be analysing long texts
df.drop(columns=['ProductName','link'], axis=1, inplace=True)
df # Check if the column is dropped

,OriginalPrice,SalePrice,Discount,Vouchers,Ratings,NoOfRatings,Comments,Media,Brand,QtySold,Favourites,SubCat1,SubCat2,SubCat3,Content
0,13.00,7.30,44,None,4.9,830,424,312,SD BIOSENSOR,16500,1700,Health & Wellness,Medical Supplies,Health Monitors,276.0
1,2.00,2.00,0,None,5.0,251,96,61,KODOMO,4200,222,"Toys, Kids & Babies",Bath & Baby Care,Baby Wipes,296.0
2,5.10,4.80,6,"$6 OFF, $12 OFF, $25 OFF",5.0,70,32,19,Dr.Tung's,648,68,Beauty & Personal Care,Oral Care,Oral Accessories,296.0
3,6.90,2.45,64,"$6 OFF, $12 OFF, $25 OFF",5.0,154,59,53,Watsons,606,76,Beauty & Personal Care,Skincare,Moisturizer,296.0
4,4.90,4.90,0,"$6 OFF, $12 OFF, $25 OFF",5.0,143,51,44,Watsons,439,96,Beauty & Personal Care,Beauty Devices & Tools,Makeup Accessories,519.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,6.25,6.25,0,"$6 OFF, $12 OFF, $25 OFF",5.0,1,0,0,durex,3,0,Health & Wellness,Sexual Wellness,Condoms,1098.0
2922,36.45,36.45,0,"$6 OFF, $12 OFF, $25 OFF",5.0,1,1,1,NATURE'S ESSENCE,1,0,Health & Wellness,Food Nutrition,Others,992.0
2923,39.90,33.85,15,"$6 OFF, $12 OFF, $25 OFF",5.0,1,1,1,KINOHIMITSU,1,0,Health & Wellness,Weight Management,Others,793.0
2924,12.00,10.20,15,"$6 OFF, $12 OFF, $25 OFF",5.0,1,0,0,LEADERS,1,1,Beauty & Personal Care,Skincare,Mask,1687.0


#### 1.3.1 Get Dummies for string columns

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2926 entries, 0 to 2925
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   OriginalPrice  2926 non-null   float64
 1   SalePrice      2926 non-null   float64
 2   Discount       2926 non-null   int64  
 3   Vouchers       2926 non-null   object 
 4   Ratings        2468 non-null   float64
 5   NoOfRatings    2926 non-null   int64  
 6   Comments       2926 non-null   int64  
 7   Media          2926 non-null   int64  
 8   Brand          2926 non-null   object 
 9   QtySold        2926 non-null   int64  
 10  Favourites     2926 non-null   int64  
 11  SubCat1        2926 non-null   object 
 12  SubCat2        2926 non-null   object 
 13  SubCat3        2926 non-null   object 
 14  Content        2926 non-null   float64
dtypes: float64(4), int64(6), object(5)
memory usage: 343.0+ KB


In [12]:
nominal = ['Vouchers', 'Brand', 'SubCat1', 'SubCat2', 'SubCat3']

In [13]:
df[df['Brand']=="None"]

,OriginalPrice,SalePrice,Discount,Vouchers,Ratings,NoOfRatings,Comments,Media,Brand,QtySold,Favourites,SubCat1,SubCat2,SubCat3,Content


In [14]:
# Apply one hot encoding to two datasets.
df1 = pd.get_dummies(df, columns=nominal, drop_first=True)

In [15]:
df1

,OriginalPrice,SalePrice,Discount,Ratings,NoOfRatings,Comments,Media,QtySold,Favourites,Content,Vouchers_None,Brand_- None -,Brand_21ST CENTURY,Brand_3M,Brand_A-DERMA,Brand_AFC,Brand_AHC,Brand_AMMELTZ,Brand_ANESSA,Brand_ASTALIFT,Brand_ATORREGE AD+,Brand_AVALON,Brand_AVEDA,Brand_Abbott,Brand_Accu-Chek,Brand_Acti-Tape,Brand_Against24,Brand_Alcon,Brand_Aleve,Brand_Alka-Seltzer,Brand_Alo Natura,Brand_Appeton,Brand_Ascensia,Brand_Audace,Brand_Audisol,Brand_Avalon Organics,Brand_Aveeno,Brand_Axe Brand,Brand_BACTISHIELD,Brand_BANANA BOAT,Brand_BANILA CO,Brand_BAUSCH + LOMB,Brand_BAUSCH AND LOMB,Brand_BIOGLAN,Brand_BLACKMORES,Brand_BONOTOX,Brand_BRAND'S,Brand_Baby Carrie,Brand_Baby Sebamed,Brand_Bactidol,Brand_Batiste,Brand_Bausch & Lomb,Brand_Beautylabo,Brand_Bekunis,Brand_Beneprotein,Brand_Berocca,Brand_Betadine,Brand_Bifesta,Brand_Bigen,Brand_Bio-Oil,Brand_Bio-essence,Brand_Biocalth,Brand_Bioderma,Brand_Biolane,Brand_Biore,Brand_Biostime,Brand_Black Gold,Brand_Bonjela,Brand_Borsch Med,Brand_Browlash,Brand_CALTRATE,Brand_CANMAKE,Brand_CEZANNE,Brand_CLEAR,Brand_CLINIQUE,Brand_CLIO,Brand_COSRX,Brand_Calcit Ultra,Brand_Candazole,Brand_Canesten,Brand_Cardiprin,Brand_Carefree,Brand_Carnation Footcare,Brand_Cath Kidston,Brand_Cationorm,Brand_Cebion,Brand_Cellapy,Brand_Centrum,Brand_Ceradan,Brand_Cetaphil,Brand_Cielo,Brand_Clean & Clear,Brand_Cloversoft,Brand_Colgate,Brand_Collagen By Watsons,Brand_Contour,Brand_Cure,Brand_Curel,Brand_DAENG GI MEO RI,Brand_DARIYA,Brand_DHC,Brand_DR.WU,Brand_DairyCare,Brand_Darlie,Brand_Dashing Diva,Brand_Decolgen,Brand_Dentiste,Brand_Dequadin,Brand_Derma Lab,Brand_Desitin,Brand_Dettol,Brand_Diane,Brand_Diane Bonheur,Brand_Disney,Brand_Doctor's Best,Brand_Dolly Wink,Brand_Dove,Brand_Dr OatCare,Brand_Dr. Bronner's,Brand_Dr. Jart+,Brand_Dr. LeWinn's,Brand_Dr.G,Brand_Dr.Tung's,Brand_Dulcolax,Brand_Dumex,Brand_Duolac,Brand_EBENE,Brand_ELANCYL Laboratoire,Brand_ENERVON,Brand_EYE MO,Brand_Eagle Brand,Brand_Eau Thermale Avene,Brand_Efamol,Brand_Ego,Brand_Ego Pharmaceuticals,Brand_Elgydium,Brand_Ellips,Brand_Embryolisse,Brand_Enfagrow,Brand_Ensure,Brand_Enterogermina,Brand_Essano,Brand_Essentiale,Brand_Estee Lauder,Brand_Eu Yan Sang,Brand_Eucerin,Brand_Eversoft,Brand_Fastum Gel Max,Brand_Follow Me,Brand_Foot Petals,Brand_For Beloved One,Brand_Forest Beauty,Brand_Freedom Cups,Brand_Freestyle,Brand_Fybogel,Brand_GATSBY,Brand_GROW,Brand_Garnier,Brand_Gaviscon,Brand_Gerber,Brand_Gillette,Brand_Ginvera,Brand_Glucerna,Brand_GreenLife,Brand_HAIR SYSTEM WATSONS,Brand_HASK,Brand_HATOMUGI,Brand_HUGGIES,Brand_Hada Labo,Brand_Hadabisei,Brand_Hansaplast,Brand_Harpic,Brand_Head & Shoulders,Brand_Heavy Rotation,Brand_Herbal Essences,Brand_Herbion,Brand_Heritage,Brand_Heroine make,Brand_Herpatch,Brand_Himalaya,Brand_Hirudoid,Brand_Hiruscar,Brand_Hisamitsu,Brand_Holistic Way,Brand_Hyland's,Brand_HypoCol,Brand_ICM Pharma,Brand_Isomil,Brand_JOHNSON'S,Brand_Johnson & Johnson,Brand_Jorubi,Brand_Juicy Couture,Brand_K-palette,Brand_KINOHIMITSU,Brand_KLab,Brand_KOBAYASHI,Brand_KODOMO,Brand_Kao,Brand_Karihome,Brand_Kate Tokyo,Brand_KinderNurture,Brand_Kirei Kirei,Brand_Kleenex,Brand_Klorane,Brand_Koolfever,Brand_Kordel's,Brand_Kose,Brand_Kose Cosmeport,Brand_L'OREAL,Brand_L'OREAL PARIS,Brand_LAB SERIES,Brand_LEADERS,Brand_LION,Brand_LUX,Brand_La Roche-Posay,Brand_Labo Labo,Brand_Lacteol Fort,Brand_Lansinoh,Brand_Laundrin',Brand_Laurier,Brand_Leftose,Brand_Liese,Brand_Lifebuoy,Brand_Lille,Brand_Listerine,Brand_LuLuLun,Brand_Lucido-l,Brand_Lysol,Brand_MARO 17,Brand_MEDIHEAL,Brand_MELANO CC,Brand_MENTHOLATUM,Brand_MINON,Brand_MOROCCANOIL,Brand_Ma Cherie,Brand_MamyPoko,Brand_Mane 'n Tail,Brand_Marc Anthony,Brand_Maybelline New York,Brand_MegRhythm,Brand_Milani Cosmetics,Brand_Minon Amino Moist,Brand_Mise-En-Scene,Brand_Moz Away,Brand_Mucosolvan,Brand_Mundicare,Brand_My Beauty Diary,Brand_NARUKO,Brand_NATURAL BRAND,Brand_NATURE'S ESSENCE,Brand_NaturVital,Brand_Naturals by Watsons,Brand_Nature's Essentials,Brand_Nature's Green,Brand_Nature's Way

## 2. Modelling
[*Jump to table of contents*](#Table-of-Contents)
### 2.1 PyCaret for Baseline Model
[*Jump to table of contents*](#Table-of-Contents)

In [16]:
# data = get_data(df1)
reg1 = setup(data = df1, target = 'QtySold', normalize = True, remove_multicollinearity = True)

,Description,Value
0,session_id,3737
1,Target,QtySold
2,Original Data,"(2926, 611)"
3,Missing Values,True
4,Numeric Features,610
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(2048, 582)"


In [ ]:
best_model = compare_models(fold=5)

IntProgress(value=0, description='Processing: ', max=94)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:24:22
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
par,Passive Aggressive Regressor,23.4196,80021.6584,177.8110,0.5134,0.9485,2.0452,0.1540
huber,Huber Regressor,22.7580,80296.4060,178.5960,0.5058,0.8885,1.6696,0.9940
llar,Lasso Least Angle Regression,26.4121,69261.8000,175.5100,0.3505,1.0119,2.4456,0.0620
knn,K Neighbors Regressor,26.2855,123090.5744,216.4414,0.3240,1.0247,1.9122,0.3860
en,Elastic Net,33.3608,108780.2111,205.1735,0.2116,1.2303,3.8560,0.0340
br,Bayesian Ridge,40.9801,29343.5036,139.9887,-0.2554,1.4654,6.1347,0.3820
lasso,Lasso Regression,41.0897,31504.7094,143.9319,-0.2853,1.5008,6.3348,0.0460
ridge,Ridge Regression,42.9379,26651.2083,137.1705,-0.3562,1.5240,6.6954,0.0560
omp,Orthogonal Matching Pursuit,32.1531,31229.4636,146.8920,-0.5521,1.2160,4.3991,0.0680
dt,Decision Tree Regressor,25.1533,90208.4218,216.7994,-1.3585,0.8800,1.1490,0.1040


In [15]:
huber = create_model ('huber')
tuned_huber = tune_model(et, n_iter =100)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,7.7833,938.0417,30.6275,0.7492,0.5355,0.5518
1,6.1588,174.9853,13.2282,0.7937,0.5584,0.5931
2,12.9294,3611.2999,60.0941,0.6203,0.5246,0.6052
3,7.9195,644.8211,25.3933,0.6330,0.4967,0.5632
4,7.7526,410.8750,20.2701,0.9246,0.5000,0.5241
5,13.9141,3805.7241,61.6906,0.3313,0.4905,0.5714
6,72.4575,850181.1319,922.0527,0.3564,0.5181,0.5535
7,5.4765,123.1886,11.0990,0.9166,0.4856,0.5670
8,6.3881,253.7231,15.9287,0.8702,0.5483,0.5934
9,22.5819,42687.0207,206.6084,0.5101,0.5632,0.6138


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:38:59
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 81.1min


KeyboardInterrupt: 

In [ ]:
bagged_et = ensemble_model(et, method = 'Bagging')

In [ ]:
boosted_et = ensemble_model(et, method = "Boosting", n_estimators = 100)